In [8]:
from pyquil.simulation.matrices import I, X, Y, Z
import numpy as np
from forest.benchmarking.operator_tools import kraus2pauli_liouville
from print_large import _print_big_matrix
import pickle
from tqdm import tqdm
import multiprocessing as mp
import random
#gateset1 [1, 9601, 8129, 3103, 2296, 0, 0, 9285, 8118, 2974, 3058, 0, 0, 10523, 9788]
#gateset2 [16192, 11846, 15055, 13880, 10476, 10481]
###rb_seq_deompose
with open('rb_seq_decompose_gateset1.pkl', 'rb') as f:
    rb_seq_decompose = pickle.load(f)
###primitive gate PTM
with open('P_PTM_gateset1.pkl', 'rb') as f:
    P_PTM = pickle.load(f)
#### 2-qubit pauli basis
pauli_basis=[np.asarray(np.kron(i,j)) for i in [I, X, Y, Z] for j in [I, X, Y, Z]]
#### 2-qubit |0> and |1>
e_state=np.kron(np.asarray([[0,0],[0,1]]),np.asarray([[0,0],[0,1]]))
g_state=np.kron(np.asarray([[1,0],[0,0]]),np.asarray([[1,0],[0,0]]))
####Density matrix to PTM
def densityop_01_2_pauli(density_op):
    #assert density_op.ndim !=4,'wrong dim'
    A=np.asarray([np.trace(density_op@i_den) for i_den in pauli_basis]).reshape(density_op.size,1)
    return A
def densityop_pauli_2_01(density_op):
    #assert density_op.shape == (len(density_op),1),'wrong dim'
    A_den=np.asarray([density_op[i_den]*pauli_basis[i_den] for i_den in range(16)])
    B_den=np.zeros((4,4))
    for i_den in A_den:
        B_den=B_den+i_den
    return B_den/4
def dephaseing_PTM(p):
    A=[np.kron(i_deph,j_deph) for i_deph in [I,Z] for j_deph in [I,Z]]
    for i_deph in [0]:
        A[i_deph] = A[i_deph]*np.sqrt(p+(1-p)/4)
    for i_deph in [1,2,3]:
        A[i_deph] = A[i_deph]*np.sqrt((1-p)/4)
#########kraus to PTM 
    return kraus2pauli_liouville(A)
def depolarizing_PTM(p):
    A=[np.kron(i_depo,j_depo) for i_depo in [I,X,Y,Z] for j_depo in [I,X,Y,Z]]
    A[0] = A[0]*np.sqrt(1-15*p/16)
    for i_depo in range(1,len(A)):
        A[i_depo]=A[i_depo]*np.sqrt(p/16)
#########kraus to PTM        
    return kraus2pauli_liouville(A)
def sequence_to_probability(_rb_seq):
####initial state    
    _state=densityop_01_2_pauli(g_state)
####depo or deph=  noise channel @ Cliff @ state
    if noise_channel==0:
        for _i in _rb_seq :
            _state=depolarizing_PTM(noise_strength)@P_PTM[_i]@_state
    elif noise_channel==1:
        for _i in _rb_seq :
            _state=dephaseing_PTM(noise_strength)@P_PTM[_i]@_state
####get [00] prob
    return np.real(densityop_pauli_2_01(_state)[0][0])



In [1]:
if __name__ == '__main__':

####### input
    rb_types=int(input('-1 rb, 0 decomp_rb, 1 prb\n'))
    max_seq= int(input('max seq \n'))
    noise_channel= int(input('depolarizing (0) or dephasing (1) \n'))
    noise_strength= float(input('noisy(0)-----(1)noiseless\n'))
####### rb gate number i    
    for i in tqdm([i for i in range(1,max_seq)],ascii=True,desc="sequence_length",dynamic_ncols=True):
####### rb sequence into probablility
        m_average=[]
        pool = mp.Pool()
        repeat_probability = pool.map(sequence_to_probability,rb_seq_decompose[i])
        pool.close()
        pool.join()

NameError: name 'dephaseing_PTM' is not defined

In [14]:
#test
if __name__ == '__main__':

####### input
    rb_types=-1

    noise_channel= 0
    noise_strength= 1
####### rb gate number i 
    reg=P_PTM[9]
    for i in [ 14, 8, 7, 2, 1, 9, 14, 8, 7, 2, 1]:
       reg= P_PTM[i]@reg
    
    _print_big_matrix(reg)


1.0  .   .   .   .   .   .   .   .   .   .   .   .   .   .   .  
 .   .   .   .   .   .  -1.0  .   .   .   .   .   .   .   .   .  
 .   .   .   .   .   .   .   .   .   .   .   .  1.0  .   .   .  
 .   .   .   .   .   .   .   .   .   .  1.0  .   .   .   .   .  
 .   .   .   .   .   .   .   .   .   .   .   .   .  -1.0  .   .  
 .   .   .   .   .   .   .   .   .   .   .  -1.0  .   .   .   .  
 .  -1.0  .   .   .   .   .   .   .   .   .   .   .   .   .   .  
 .   .   .   .   .   .   .  -1.0  .   .   .   .   .   .   .   .  
 .   .   .   .   .   .   .   .   .   .   .   .   .   .   .  1.0 
 .   .   .   .   .   .   .   .   .  -1.0  .   .   .   .   .   .  
 .   .   .  1.0  .   .   .   .   .   .   .   .   .   .   .   .  
 .   .   .   .   .  -1.0  .   .   .   .   .   .   .   .   .   .  
 .   .  1.0  .   .   .   .   .   .   .   .   .   .   .   .   .  
 .   .   .   .  -1.0  .   .   .   .   .   .   .   .   .   .   .  
 .   .   .   .   .   .   .   .   .   .   .   .   .   .  1.0  .  
 .   .   .   .   

In [2]:
from tqdm import trange
import pickle
import numpy as np
import multiprocessing as mp
with open('Cliff_index_gateset1.pkl', 'rb') as f:
    old_Cliff_index_gateset = pickle.load(f)
with open('Cliff_kruas.pkl', 'rb') as f:
    Cliff_kruas = pickle.load(f)
with open('P_kruas_gateset1.pkl', 'rb') as f:
    P_kruas_gateset = pickle.load(f)

def reindex(i):
### first matrix
    _reg=np.eye(4)
### else matrix    
    for j in old_Cliff_index_gateset[i]:
        _reg=P_kruas_gateset[j]@_reg
### new [j]   = old [i] 
    for j in range(11520):
        if np.isclose(np.trace(_reg@np.conj(Cliff_kruas[j]).T)/4,1):
            new_Cliff_index_gateset[j]=old_Cliff_index_gateset[i]   
            break
if __name__ == '__main__':
    ###reindex
    new_Cliff_index_gateset=[False]*11520
    pool = mp.Pool()
    pool.map_async(reindex,[i for i in range(11520)])
    pool.close()
    pool.join()

    with open('Cliff_index_gateset1.pkl', 'wb') as f:
        pickle.dump(new_Cliff_index_gateset, f)
